In [ ]:
import os
import json
from tqdm import tqdm
import copy

In [ ]:
data_path = "../normal_data/wplice/"
save_path = "../normal_data/short_wplice/"
train_path = data_path + "train_data.json"
valid_path = data_path + "valid_data.json"
test_path = data_path + "test_data.json"

In [ ]:
train_data = json.load(open(train_path, "r", encoding = "utf-8"))
valid_data = json.load(open(valid_path, "r", encoding = "utf-8"))
test_data = json.load(open(test_path, "r", encoding = "utf-8"))

In [ ]:
def shorter(data, around = 100, max_char_num = 5000):
    new_data = []
    for sample in tqdm(data):
        start_idx, end_idx = 1 << 31, 0
        for ent in sample["entity_list"]:
            char_span = ent["char_span"]
            start_idx = min(start_idx, char_span[0])
            end_idx = max(end_idx, char_span[1])
        start_idx = max(start_idx - around, 0)
        new_text = sample["text"][start_idx: end_idx + around]
        if len(new_text) > max_char_num:
            continue
        new_sample = copy.deepcopy(sample)
        new_sample["text"] = new_text
        for ent in new_sample["entity_list"]:
            char_span = ent["char_span"]
            char_span[0] -= start_idx
            char_span[1] -= start_idx
            assert new_text[char_span[0]:char_span[1]] == ent["text"]
        new_data.append(new_sample)
    return new_data

In [ ]:
short_train_data = shorter(train_data, 200, 1000)
short_valid_data = shorter(valid_data, 200, 1000)
short_test_data = shorter(test_data, 200, 1000)

In [ ]:
# fix id
def set_ids(data, data_type):
    for idx, sample in enumerate(data):
        sample["id"] = "{}_{}".format(data_type, idx)

In [ ]:
set_ids(short_train_data, "train")
set_ids(short_valid_data, "valid")
set_ids(short_test_data, "test")

In [ ]:
if not os.path.exists(save_path):
    os.makedirs(save_path)

json.dump(short_train_data, open(save_path + "train_data.json", "w", encoding = "utf-8"), ensure_ascii = False)
json.dump(short_valid_data, open(save_path + "valid_data.json", "w", encoding = "utf-8"), ensure_ascii = False)
json.dump(short_test_data, open(save_path + "test_data.json", "w", encoding = "utf-8"), ensure_ascii = False)